In [123]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import lightgbm as lgb

In [125]:
df1 = pd.read_csv("ford.csv")
df2 = pd.read_csv("audi.csv")
df3 = pd.read_csv("bmw.csv")
df4 = pd.read_csv("mercedes.csv")
df5 = pd.read_csv("toyota.csv")
df6 = pd.read_csv("hyundai.csv")
df = pd.concat([df1, df2, df3, df4, df5, df6])
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,Fiesta,2017,12000,Automatic,15944,Petrol,150.0,57.7,1.0,NaN
1,Focus,2018,14000,Manual,9083,Petrol,150.0,57.7,1.0,NaN
2,Focus,2017,13000,Manual,12456,Petrol,150.0,57.7,1.0,NaN
3,Fiesta,2019,17500,Manual,10460,Petrol,145.0,40.3,1.5,NaN
4,Fiesta,2019,16500,Automatic,1482,Petrol,145.0,48.7,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
4855,I30,2016,8680,Manual,25906,Diesel,NaN,78.4,1.6,0.0
4856,I40,2015,7830,Manual,59508,Diesel,NaN,65.7,1.7,30.0
4857,I10,2017,6830,Manual,13810,Petrol,NaN,60.1,1.0,20.0
4858,Tucson,2018,13994,Manual,23313,Petrol,NaN,44.8,1.6,145.0


In [126]:
from sklearn.preprocessing import LabelEncoder
Numerics = LabelEncoder()

In [128]:
inputs = df.drop(['tax(£)', 'tax', 'engineSize'], axis='columns')
target = df.price
target

0       12000
1       14000
2       13000
3       17500
4       16500
        ...  
4855     8680
4856     7830
4857     6830
4858    13994
4859    15999
Name: price, Length: 64131, dtype: int64

In [129]:
inputs['model_n']=Numerics.fit_transform(inputs['model'])
inputs['transmission_n']=Numerics.fit_transform(inputs['transmission'])
inputs['fueltype_n']=Numerics.fit_transform(inputs['fuelType'])
inputs


,model,year,price,transmission,mileage,fuelType,mpg,model_n,transmission_n,fueltype_n
0,Fiesta,2017,12000,Automatic,15944,Petrol,57.7,38,0,4
1,Focus,2018,14000,Manual,9083,Petrol,57.7,39,1,4
2,Focus,2017,13000,Manual,12456,Petrol,57.7,39,1,4
3,Fiesta,2019,17500,Manual,10460,Petrol,40.3,38,1,4
4,Fiesta,2019,16500,Automatic,1482,Petrol,48.7,38,0,4
...,...,...,...,...,...,...,...,...,...,...
4855,I30,2016,8680,Manual,25906,Diesel,78.4,56,1,0
4856,I40,2015,7830,Manual,59508,Diesel,65.7,57,1,0
4857,I10,2017,6830,Manual,13810,Petrol,60.1,54,1,4
4858,Tucson,2018,13994,Manual,23313,Petrol,44.8,111,1,4


In [130]:
inputs_n = inputs.drop(['price', 'model', 'transmission', 'fuelType'], axis='columns')
inputs_n

,year,mileage,mpg,model_n,transmission_n,fueltype_n
0,2017,15944,57.7,38,0,4
1,2018,9083,57.7,39,1,4
2,2017,12456,57.7,39,1,4
3,2019,10460,40.3,38,1,4
4,2019,1482,48.7,38,0,4
...,...,...,...,...,...,...
4855,2016,25906,78.4,56,1,0
4856,2015,59508,65.7,57,1,0
4857,2017,13810,60.1,54,1,4
4858,2018,23313,44.8,111,1,4


In [131]:
model = linear_model.LinearRegression()

In [132]:
model.fit(inputs_n.values, target)

LinearRegression()

In [133]:
linear_r2 = model.score(inputs_n.values, target)
linear_r2

0.43071126452051123

In [134]:
pred=model.predict([[2016, 20000, 43.5, 14, 0, 0]])
pred

array([20725.47071514])

In [135]:
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    n_estimators=100,
    reg_lambda=1.0
)

lgb_model.fit(inputs_n, target)

lgb_r2 = lgb_model.score(inputs_n.values, target)
lgb_r2

0.9449518198000206

In [137]:
pred=lgb_model.predict([[2016, 11472, 57.6, 111, 0, 0]])
pred

array([15305.24119662])